In [2]:
# !pip install lxml

from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import re
from time import sleep
from lxml import etree

pd.set_option('display.max_columns', None)

### Brief intro
The web I want to webscrap is Vinissimus.com, it is an Spanish online specialized store of wines and liquors containing information about 7093 Spanish wines, 6620 Italian wines and 2495 French wines. <br>
I will focus my efforts in first scraping the info about the Spanish wines. <br>
Due to the structure of the webpage, first I need to obtain the links of each wine, and then from each link obtain all the information I need.

### Getting the links for each wine (first page)

In [2]:
# en español "https://www.vinissimus.com/es/region/espana/"

url = "https://www.vinissimus.com/en/region/spain/?content=%5B%22region%22%2C%22espana%22%5D" # english
response = requests.get(url)
response.status_code   

200

In [3]:
soup = BeautifulSoup(response.content, "html.parser")

In [ ]:
# Obtaining the links

links_page_one = []
for a in soup.find_all("div", attrs={"class": "details"}):
    links_page_one.append(a.find("a").get("href"))
    
links_page_one

In [ ]:
# Add the beginning of the link to all links

missing_part = "https://www.vinissimus.com"

links_page_one_compl = [missing_part + l for l in links_page_one]
links_page_one_compl

### Getting the links for each wine (rest of the pages)
There are 198 pages. These are the urls for the pages 2, 3, 4 and 198 (spanish):
- https://www.vinissimus.com/es/region/espana/?sort_by=%5B%7B%22field%22%3A%22opciones.in_stock%22%2C%22order%22%3A%22asc%22%7D%2C%7B%22field%22%3A%22ranking%22%2C%22order%22%3A%22asc%22%7D%5D&cursor=36
- https://www.vinissimus.com/es/region/espana/?sort_by=%5B%7B%22field%22%3A%22opciones.in_stock%22%2C%22order%22%3A%22asc%22%7D%2C%7B%22field%22%3A%22ranking%22%2C%22order%22%3A%22asc%22%7D%5D&cursor=72 
- https://www.vinissimus.com/es/region/espana/?sort_by=%5B%7B%22field%22%3A%22opciones.in_stock%22%2C%22order%22%3A%22asc%22%7D%2C%7B%22field%22%3A%22ranking%22%2C%22order%22%3A%22asc%22%7D%5D&cursor=108 
- https://www.vinissimus.com/es/region/espana/?sort_by=%5B%7B%22field%22%3A%22opciones.in_stock%22%2C%22order%22%3A%22asc%22%7D%2C%7B%22field%22%3A%22ranking%22%2C%22order%22%3A%22asc%22%7D%5D&cursor=7092

English: 
- https://www.vinissimus.com/en/region/spain/?sort_by=%5B%7B%22field%22%3A%22opciones.in_stock%22%2C%22order%22%3A%22asc%22%7D%2C%7B%22field%22%3A%22ranking%22%2C%22order%22%3A%22asc%22%7D%5D&cursor=36
- https://www.vinissimus.com/en/region/spain/?sort_by=%5B%7B%22field%22%3A%22opciones.in_stock%22%2C%22order%22%3A%22asc%22%7D%2C%7B%22field%22%3A%22ranking%22%2C%22order%22%3A%22asc%22%7D%5D&cursor=72

The last figure increases from 36 to 36.

In [13]:
# Obtain the total number of wines 

number_string = soup.find_all("span", attrs={"class":"total-count"})[0].get_text()
num = ""
for c in number_string:
    if c.isdigit():
        num = num + c
total_number = int(num)
total_number

7099

In [14]:
pages = list(range (36, total_number, 36))
pages

[36,
 72,
 108,
 144,
 180,
 216,
 252,
 288,
 324,
 360,
 396,
 432,
 468,
 504,
 540,
 576,
 612,
 648,
 684,
 720,
 756,
 792,
 828,
 864,
 900,
 936,
 972,
 1008,
 1044,
 1080,
 1116,
 1152,
 1188,
 1224,
 1260,
 1296,
 1332,
 1368,
 1404,
 1440,
 1476,
 1512,
 1548,
 1584,
 1620,
 1656,
 1692,
 1728,
 1764,
 1800,
 1836,
 1872,
 1908,
 1944,
 1980,
 2016,
 2052,
 2088,
 2124,
 2160,
 2196,
 2232,
 2268,
 2304,
 2340,
 2376,
 2412,
 2448,
 2484,
 2520,
 2556,
 2592,
 2628,
 2664,
 2700,
 2736,
 2772,
 2808,
 2844,
 2880,
 2916,
 2952,
 2988,
 3024,
 3060,
 3096,
 3132,
 3168,
 3204,
 3240,
 3276,
 3312,
 3348,
 3384,
 3420,
 3456,
 3492,
 3528,
 3564,
 3600,
 3636,
 3672,
 3708,
 3744,
 3780,
 3816,
 3852,
 3888,
 3924,
 3960,
 3996,
 4032,
 4068,
 4104,
 4140,
 4176,
 4212,
 4248,
 4284,
 4320,
 4356,
 4392,
 4428,
 4464,
 4500,
 4536,
 4572,
 4608,
 4644,
 4680,
 4716,
 4752,
 4788,
 4824,
 4860,
 4896,
 4932,
 4968,
 5004,
 5040,
 5076,
 5112,
 5148,
 5184,
 5220,
 5256,
 5292,


In [ ]:
links_rest_pages = []

for page in pages:
    # request = requests.get(f"https://www.vinissimus.com/es/region/espana/?sort_by=%5B%7B%22field%22%3A%22opciones.in_stock%22%2C%22order%22%3A%22asc%22%7D%2C%7B%22field%22%3A%22ranking%22%2C%22order%22%3A%22asc%22%7D%5D&cursor={page}")
    request = requests.get(f"https://www.vinissimus.com/en/region/spain/?sort_by=%5B%7B%22field%22%3A%22opciones.in_stock%22%2C%22order%22%3A%22asc%22%7D%2C%7B%22field%22%3A%22ranking%22%2C%22order%22%3A%22asc%22%7D%5D&cursor={page}")
    soup = BeautifulSoup(request.content, 'html.parser')
    for a in soup.find_all("div", attrs={"class": "details"}):
        links_rest_pages.append(a.find("a").get("href"))
    wait_time = np.random.randint(1,5)
    sleep(wait_time)

links_rest_pages

In [ ]:
len(links_rest_pages)

In [ ]:
# Add the beginning of the link to all links

missing_part = "https://www.vinissimus.com"

links_rest_pages_compl = [missing_part + l for l in links_rest_pages]
links_rest_pages_compl

In [ ]:
# Join the two lists  

links = links_page_one_compl + links_rest_pages_compl
links

In [ ]:
len(links)

In [ ]:
df = pd.DataFrame(links)
df

In [ ]:
df.to_csv("links.csv")

### Get the info from each wine (first wine)

In [62]:
url = "https://www.vinissimus.com/en/wine/ultreia-saint-jacques/"
response = requests.get(url)
response.status_code

200

In [63]:
soup = BeautifulSoup(response.content, "html.parser")

In [ ]:
# links_page_one = []
# for a in soup.find_all("div", attrs={"class": "details"}):
#     links_page_one.append(a.find("a").get("href"))
    
# links_page_one

In [ ]:
soup.find_all("div", attrs={"class": "product-title"})

In [ ]:
name = []
name.append(soup.find_all("div", attrs={"class": "product-title"})[0].get_text())
name

In [ ]:
price = []
price.append(soup.find_all("p", attrs={"class": "price uniq large"})[0].get_text())
price

In [ ]:
soup.find_all("p", attrs={"class": "price uniq large"})

In [ ]:
volume = []
volume.append(soup.find_all("div", attrs={"class": "quantity-widget large has-0-flags"})[0].select("span", attrs={"class": "unit-name  large"})[2].get_text())
volume

In [ ]:
for a in soup.find_all("table", attrs={"class": "styles_table__jWPKo info-grid technical-info"}):
    for a in (a.select("td")):
        print(a.get_text())

In [ ]:
wine_type = []
region = []
grapes = []
producer = []
alergens = []
alcohol = []
production = []
material = []
aging_period = []
casks_age = []
casks_wood = []
winemake = []
vineyard_age = []
soil_type = []
climate = []
orientation = []
surface = []

spec_sheet = [wine_type, region, grapes, producer, alergens, alcohol, production]
winemaking = [material, aging_period, casks_age, casks_wood, winemake]
vineyards = [vineyard_age, soil_type, climate, orientation, surface]

technical_info = [spec_sheet, winemaking, vineyards]

In [ ]:
for info in range(0, len(technical_info)):
    for info2 in range(0, len(technical_info[info])):
        technical_info[info][info2].append(soup.find_all("table", attrs={"class": "styles_table__jWPKo info-grid technical-info"})[info].select("td")[info2].get_text())
technical_info

In [ ]:
len(technical_info)

In [ ]:
# types = []
# types.append(soup.find_all("table", attrs={"class": "styles_table__jWPKo info-grid technical-info"})[0].select("td")[0].get_text())
# types

In [ ]:
# region = []
# region.append(soup.find_all("table", attrs={"class": "styles_table__jWPKo info-grid technical-info"})[0].select("td")[1].get_text())
# region

In [ ]:
image = []
image.append(soup.find_all("div", attrs={"class": "product-image desktop"})[0].find("img").get("src"))
image

In [ ]:
wine_descr = []
tasting_descr = []
conditions_descr = []
winemake_descr = []
vineyards_descr = []

broad_text = [wine_descr, tasting_descr, conditions_descr, winemake_descr, vineyards_descr]

for descr in range(0, len(broad_text)):
    broad_text[descr].append(soup.find_all("div", attrs={"class": "sanitize-html"})[descr].get_text())
broad_text    

In [ ]:
serve_temp = []
serve_temp.append(soup.find_all("div", attrs={"style": "display: flex; justify-content: center; flex-direction: column;"})[0].get_text())
serve_temp

In [ ]:
food_pairing = []
food_pairing.append(soup.find_all("div", attrs={"style": "display: flex; flex-wrap: wrap; gap: 10px; line-height: 18px; font-size: 11px; text-transform: uppercase;"})[0].get_text())
food_pairing

In [ ]:
# no he podido sacar la otra parte de maridajes

# for a in soup.find_all("div", attrs={"class": "content"}):
#     print(a.select("p", attrs={"style", "margin-top: 20px;"}))

In [ ]:
customer_reviews = []
customer_reviews.append(soup.find_all("div", attrs={"class": "styles_firstBlock__NtfrI"})[1].get_text())
customer_reviews

In [ ]:
customer_reviews_number = []
customer_reviews_number.append(soup.find_all("div", attrs={"class": "opinion-widget rating"})[0].find("span", attrs={"class":"styles_numOpinions__t_p9L"}).get_text())
customer_reviews_number

In [ ]:
scores = []
for a in soup.find_all("table", attrs={"class": "styles_table__jWPKo info-grid score-awards"}):
    for b in a.find_all("td"):
        scores.append(b.get_text())
scores    

In [ ]:
# Since taking the average seems complicated, for now just take the last score 
## (Although the average would be more complete, I don't want to spend too much time in this
## and the and the last score is still very relevant information

last_score = []
last_score.append(soup.find_all("table", attrs={"class": "styles_table__jWPKo info-grid score-awards"})[0].find("td").get_text())
last_score

In [ ]:
soup.find_all("table", attrs={"class": "styles_table__jWPKo info-grid score-awards"})[0].get_text()

In [ ]:
soup.find_all("table", attrs={"class": "styles_table__jWPKo info-grid score-awards"})

In [ ]:
last_score_year = []
last_score_year.append(soup.find_all("table", attrs={"class": "styles_table__jWPKo info-grid score-awards"})[0].find("th").get_text())
last_score_year

### Get the info from every wine

In [ ]:
# Function for obtaining all the info (with technical info)

def obtain_wine_info(soup):
    
    # Define lists for each feature I want to obtain
    name = []
    price = []
    volume = []
    image = []
    
    wine_type = []
    region = []
    grapes = []
    producer = []
    alergens = []
    alcohol = []
    production = []
    material = []
    aging_period = []
    casks_age = []
    casks_wood = []
    winemake = []
    vineyard_age = []
    soil_type = []
    climate = []
    orientation = []
    surface = []

    spec_sheet = [wine_type, region, grapes, producer, alergens, alcohol, production]
    winemaking = [material, aging_period, casks_age, casks_wood, winemake]
    vineyards = [vineyard_age, soil_type, climate, orientation, surface]

    technical_info = [spec_sheet, winemaking, vineyards]
    
    wine_descr = []
    tasting_descr = []
    conditions_descr = []
    winemake_descr = []
    vineyards_descr = []

    broad_text = [wine_descr, tasting_descr, conditions_descr, winemake_descr, vineyards_descr]
    
    serve_temp = []
    food_pairing = []
    customer_reviews = []
    customer_reviews_number = []
    last_score = []
    last_score_year = []
    
    # Assing elements to each list

    name.append(soup.find_all("div", attrs={"class": "product-title"})[0].get_text()) 
    price.append(soup.find_all("p", attrs={"class": "price uniq large"})[0].get_text()) 
    volume.append(soup.find_all("div", attrs={"class": "quantity-widget large has-0-flags"})[0].select("span", attrs={"class": "unit-name  large"})[2].get_text()) 
    image.append(soup.find_all("div", attrs={"class": "product-image desktop"})[0].find("img").get("src")) 
    
    for info in range(0, len(technical_info)):
        for info2 in range(0, len(technical_info[info])):
            technical_info[info][info2].append(soup.find_all("table", attrs={"class": "styles_table__jWPKo info-grid technical-info"})[info].select("td")[info2].get_text())
        
    for descr in range(0, len(broad_text)):
        broad_text[descr].append(soup.find_all("div", attrs={"class": "sanitize-html"})[descr].get_text())

    serve_temp.append(soup.find_all("div", attrs={"style": "display: flex; justify-content: center; flex-direction: column;"})[0].get_text())
    food_pairing.append(soup.find_all("div", attrs={"style": "display: flex; flex-wrap: wrap; gap: 10px; line-height: 18px; font-size: 11px; text-transform: uppercase;"})[0].get_text())
    customer_reviews.append(soup.find_all("div", attrs={"class": "styles_firstBlock__NtfrI"})[1].get_text())
    customer_reviews_number.append(soup.find_all("div", attrs={"class": "opinion-widget rating"})[0].find("span", attrs={"class":"styles_numOpinions__t_p9L"}).get_text())
    last_score.append(soup.find_all("table", attrs={"class": "styles_table__jWPKo info-grid score-awards"})[0].find("td").get_text())
    last_score_year.append(soup.find_all("table", attrs={"class": "styles_table__jWPKo info-grid score-awards"})[0].find("th").get_text())
    
    # Create dictionary that stores all elements
    
    dct = {"name": name, "price": price, "bottle": volume, "image": image, "type": wine_type, "region": region,
           "grapes": grapes, "producer": producer, "alergens": alergens, "alcohol_content": alcohol,
           "production": production, "material": material, "aging_period": aging_period, "casks_age": casks_age,
           "casks_wood": casks_wood, "more_winemaking_info": winemake, "vineyard_age": vineyard_age,
           "soil_type":soil_type, "climate": climate, "orientation": orientation, "surface": surface,
           "wine_description": wine_descr, "tasting": tasting_descr, "serving_storage_conditions": conditions_descr,
           "winemaking_info": winemake_descr, "vineyards": vineyards_descr, "serve_temperature": serve_temp, 
           "food_pairing": food_pairing, "customer_reviews": customer_reviews,
           "customer_reviews_number": customer_reviews_number, "last_score": last_score,
           "last_score_year": last_score_year}
   
    return dct
    

In [53]:
# Functions for dealing with errors when there is no info

def get_element(lst, index):
    try:
        lst[index]
    except IndexError:
        return ""
    else:
        return lst[index].get_text()
    
def get_price(lst, index):
    try:
        element = lst[index].get_text()
    except IndexError:
        element = soup.find_all("div", attrs={"class": "pprice"})[0].find("p").get_text()
    
    return element

def get_volume(soup):
    try:
        element = soup.find_all("div", attrs={"class": "quantity-widget large has-0-flags"})[0].select("span", attrs={"class": "unit-name  large"})[2].get_text()
    except IndexError:
        element = soup.find_all("div", attrs={"class": "pprice"})[0].find_all("span", attrs={"class":"unit-name red large"})[0].get_text()
    
    return element

def get_technical_info(lst, index1, index2):
    try:
        element = lst[index1]
    except IndexError:
        return ""
    else:
        try:
            td_element = element.select("td")[index2]
        except IndexError:
            return ""
        else:
            return td_element.get_text()
    
def get_score(lst, index, score_name):
    try:
        element = lst[index]
    except IndexError:
        return ""
    else:
        try: 
            td_element = element.find("td", attrs={"title": score_name})
        except AttributeError:
            return ""
        else:
            if td_element is not None:
                return td_element.get_text()
            else:
                return ""

def get_score_year(lst, index):
    try:
        lst[index]
    except IndexError:
        return ""
    else:
        return lst[index].find("th").get_text()    

In [ ]:
get(soup.find_all("div", attrs={"style": "display: flex; justify-content: center; flex-direction: column;"}), 0)

In [22]:
# Function for obtaining LESS info (many wines don't have all the info on technical specifications)
# + functions to deal with errors

def obtain_wine_info(soup):
    
    # Define lists for each feature I want to obtain
    name = []
    price = []
    volume = []
    image = []
    
    wine_type = []
    region = []
    grapes = []
    producer = []
    alergens = []
    alcohol = []

    spec_sheet = [wine_type, region, grapes, producer, alergens, alcohol]
    
    wine_descr = []
    tasting_descr = []
    conditions_descr = []
    winemake_descr = []
    vineyards_descr = []

    broad_text = [wine_descr, tasting_descr, conditions_descr, winemake_descr, vineyards_descr]
    
    serve_temp = []
    food_pairing = []
    customer_reviews = []
    customer_reviews_number = []
    last_score_parker = []
    last_score_penin = []
    last_score_year = []
    
    # Assing elements to each list

    name.append(soup.find_all("div", attrs={"class": "product-title"})[0].get_text()) 
    price.append(soup.find_all("p", attrs={"class": "price uniq large"})[0].get_text()) 
    volume.append(soup.find_all("div", attrs={"class": "quantity-widget large has-0-flags"})[0].select("span", attrs={"class": "unit-name  large"})[2].get_text()) 
    image.append(soup.find_all("div", attrs={"class": "product-image desktop"})[0].find("img").get("src")) 
    
    for info in range(0, len(spec_sheet)):
        spec_sheet[info].append(get(soup.find_all("table", attrs={"class": "styles_table__jWPKo info-grid technical-info"})[0].select("td"), info))
        
    for descr in range(0, len(broad_text)):
        broad_text[descr].append(get(soup.find_all("div", attrs={"class": "sanitize-html"}), descr))

    serve_temp.append(get(soup.find_all("div", attrs={"style": "display: flex; justify-content: center; flex-direction: column;"}), 0))
    food_pairing.append(get(soup.find_all("div", attrs={"style": "display: flex; flex-wrap: wrap; gap: 10px; line-height: 18px; font-size: 11px; text-transform: uppercase;"}), 0))    
    customer_reviews.append(soup.find_all("div", attrs={"class": "styles_firstBlock__NtfrI"})[1].get_text())
    customer_reviews_number.append(soup.find_all("div", attrs={"class": "opinion-widget rating"})[0].find("span", attrs={"class":"styles_numOpinions__t_p9L"}).get_text())
    last_score_parker.append(get_score(soup.find_all("table", attrs={"class": "styles_table__jWPKo info-grid score-awards"}), 0, "Parker"))
    last_score_penin.append(get_score(soup.find_all("table", attrs={"class": "styles_table__jWPKo info-grid score-awards"}), 0, "Peñín"))
    last_score_year.append(get_score_year(soup.find_all("table", attrs={"class": "styles_table__jWPKo info-grid score-awards"}), 0))
    
    # Create dictionary that stores all elements
    
    dct = {"name": name, "price": price, "bottle": volume, "image": image, "type": wine_type, "region": region,
           "grapes": grapes, "producer": producer, "alergens": alergens, "alcohol_content": alcohol,
           "wine_description": wine_descr, "tasting": tasting_descr, "serving_storage_conditions": conditions_descr,
           "winemaking_info": winemake_descr, "vineyards": vineyards_descr, "serve_temperature": serve_temp, 
           "food_pairing": food_pairing, "customer_reviews": customer_reviews,
           "customer_reviews_number": customer_reviews_number, "last_score_parker": last_score_parker,
           "last_score_penin": last_score_penin, "last_score_year": last_score_year}
   
    return dct
    

In [99]:
# Function for obtaining all the info (again with technical spec + exception handling)

def obtain_wine_info(soup):
    
    # Define lists for each feature I want to obtain
    name = []
    price = []
    volume = []
    image = []
    agriculture = []
    
    wine_type = []
    region = []
    grapes = []
    producer = []
    alergens = []
    alcohol = []
    production = []
    material = []
    aging_period = []
    casks_age = []
    casks_wood = []
    winemake = []
    vineyard_age = []
    soil_type = []
    climate = []
    orientation = []
    surface = []

    spec_sheet = [wine_type, region, grapes, producer, alergens, alcohol, production]
    winemaking = [material, aging_period, casks_age, casks_wood, winemake]
    vineyards = [vineyard_age, soil_type, climate, orientation, surface]
    technical_info = [spec_sheet, winemaking, vineyards]
    
    wine_descr = []
    tasting_descr = []
    conditions_descr = []
    winemake_descr = []
    vineyards_descr = []
    broad_text = [wine_descr, tasting_descr, conditions_descr, winemake_descr, vineyards_descr]
    
    serve_temp = []
    food_pairing = []
    customer_reviews = []
    customer_reviews_number = []
    last_score_parker = []
    last_score_penin = []
    last_score_year = []
    
    # Assing elements to each list
    name.append(soup.find_all("div", attrs={"class": "product-title"})[0].get_text()) 
    price.append(get_price(soup.find_all("p", attrs={"class": "price uniq large"}), 0)) 
    volume.append(get_volume(soup))
    image.append(soup.find_all("div", attrs={"class": "product-image desktop"})[0].find("img").get("src")) 
    agriculture.append(get(soup.find_all("div", attrs={"class": "badge-with-label-text large"}), 0))
    
    for info1 in range(0, len(technical_info)):
        for info2 in range(0, len(technical_info[info1])):
            technical_info[info1][info2].append(get_technical_info(soup.find_all("table", attrs={"class": "styles_table__jWPKo info-grid technical-info"}), info1, info2))
    
    for descr in range(0, len(broad_text)):
        broad_text[descr].append(get(soup.find_all("div", attrs={"class": "sanitize-html"}), descr))

    serve_temp.append(get(soup.find_all("div", attrs={"style": "display: flex; justify-content: center; flex-direction: column;"}), 0))
    food_pairing.append(get(soup.find_all("div", attrs={"style": "display: flex; flex-wrap: wrap; gap: 10px; line-height: 18px; font-size: 11px; text-transform: uppercase;"}), 0))    
    customer_reviews.append(soup.find_all("div", attrs={"class": "styles_firstBlock__NtfrI"})[1].get_text())
    customer_reviews_number.append(soup.find_all("div", attrs={"class": "opinion-widget rating"})[0].find("span", attrs={"class":"styles_numOpinions__t_p9L"}).get_text())
    last_score_parker.append(get_score(soup.find_all("table", attrs={"class": "styles_table__jWPKo info-grid score-awards"}), 0, "Parker"))
    last_score_penin.append(get_score(soup.find_all("table", attrs={"class": "styles_table__jWPKo info-grid score-awards"}), 0, "Peñín"))
    last_score_year.append(get_score_year(soup.find_all("table", attrs={"class": "styles_table__jWPKo info-grid score-awards"}), 0))
        
    # Create dictionary that stores all elements
    dct = {
        "name": name,
        "price": price,
        "bottle": volume,
        "image": image,
        "type_agriculture": agriculture,
        "type": wine_type,
        "region": region,
        "grapes": grapes,
        "producer": producer,
        "alergens": alergens,
        "alcohol_content": alcohol,
        "production": production,
        "material": material,
        "aging_period": aging_period,
        "casks_age": casks_age,
        "casks_wood": casks_wood,
        "more_winemaking_info": winemake,
        "vineyard_age": vineyard_age,
        "soil_type": soil_type,
        "climate": climate,
        "orientation": orientation,
        "surface": surface,
        "wine_description": wine_descr,
        "tasting": tasting_descr,
        "serving_storage_conditions": conditions_descr,
        "winemaking_info": winemake_descr,
        "vineyards": vineyards_descr,
        "serve_temperature": serve_temp,
        "food_pairing": food_pairing,
        "customer_reviews": customer_reviews,
        "customer_reviews_number": customer_reviews_number,
        "last_score_parker": last_score_parker,
        "last_score_penin": last_score_penin,
        "last_score_year": last_score_year
    }

    return dct

In [5]:
links = pd.read_csv("links.csv")
links = list(links["0"])
links

['https://www.vinissimus.com/en/wine/ultreia-saint-jacques/',
 'https://www.vinissimus.com/en/wine/blanc-pescador/',
 'https://www.vinissimus.com/en/wine/terras-gauda/',
 'https://www.vinissimus.com/en/wine/coto-de-imaz-reserva/',
 'https://www.vinissimus.com/en/wine/rene-barbier-vina-augusta-semi-dulce/',
 'https://www.vinissimus.com/en/wine/el-coto-crianza/',
 'https://www.vinissimus.com/en/wine/vina-tondonia-reserva/',
 'https://www.vinissimus.com/en/wine/vina-ardanza-reserva/',
 'https://www.vinissimus.com/en/wine/aalto/',
 'https://www.vinissimus.com/en/wine/anima-negra-an-2/',
 'https://www.vinissimus.com/en/cava/rigol-brut-nature/',
 'https://www.vinissimus.com/en/wine/pago-de-carraovejas/',
 'https://www.vinissimus.com/en/cava/freixenet-carta-nevada-semisec/',
 'https://www.vinissimus.com/en/wine/ribas-rosat/',
 'https://www.vinissimus.com/en/cava/freixenet-reserva-brut-nature/',
 'https://www.vinissimus.com/en/cava/vall-dolina-reserva-brut-nature/',
 'https://www.vinissimus.co

In [42]:
link = links[1]

request = requests.get(link)
soup = BeautifulSoup(request.content, 'html.parser')

In [95]:
info_dct = obtain_wine_info(soup)
new_df = pd.DataFrame.from_dict(info_dct)
new_df["link"] = link
new_df

,name,price,bottle,image,biodynamic,type,region,grapes,producer,alergens,alcohol_content,production,material,aging_period,casks_age,casks_wood,more_winemaking_info,vineyard_age,soil_type,climate,orientation,surface,wine_description,tasting,serving_storage_conditions,winemaking_info,vineyards,serve_temperature,food_pairing,customer_reviews,customer_reviews_number,last_score_parker,last_score_penin,last_score_year,link
0,Can Sumoi Xarel·lo 2022,£15.50,/ 0.75 L btl,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,BIODYNAMIC,White wine,Penedès (Spain),100% Xarel·lo,Can Sumoi,Contains sulfites,12.5%,"30,000 bottles",50 years,Calcareous-clay,Mediterranean,600.00 meters,,,,,,,Driving around Penedès in the search of the es...,,,,,,SaladsRiceSeafoodFishSemi-cured cheeses,4.6/5,11 reviews,92 PK,,2021,https://www.vinissimus.com/en/wine/can-sumoi-x...


In [ ]:
# Create loop to webscrap all pages

wine_data = pd.DataFrame()

for link in links:
    request = requests.get(link)
    soup = BeautifulSoup(request.content, 'html.parser')
    info_dct = obtain_wine_info(soup)
    new_df = pd.DataFrame.from_dict(info_dct)
    wine_data = pd.concat([wine_data, new_df])    
    wait_time = np.random.randint(1,5)
    print(wait_time)
    sleep(wait_time)

wine_data

In [ ]:
# Save the csv

wine_data.to_csv("wine_data.csv")

In [ ]:
wine_data

#### Price

In [48]:
link = "https://www.vinissimus.com/en/wine/marques-de-caceres-crianza/"

request = requests.get(link)
soup = BeautifulSoup(request.content, 'html.parser')

In [68]:
def get_price(lst, index):
    try:
        element = lst[index].get_text()
    except IndexError:
        element = soup.find_all("div", attrs={"class": "pprice"})[0].find("p").get_text()
    
    return element

In [69]:
price = []
price.append(get_price(soup.find_all("p", attrs={"class": "price uniq large"}), 0)) 
price

['£15.50']

In [65]:
soup.find_all("div", attrs={"class": "pprice"})[0].find("p").get_text()

'£15.50'

#### Volume

In [78]:
soup.find_all("div", attrs={"class": "pprice"})[0].find_all("span", attrs={"class":"unit-name red large"})[0].get_text()

' / 0.75 L btl '

In [91]:
def get_volume(soup):
    try:
        element = soup.find_all("div", attrs={"class": "quantity-widget large has-0-flags"})[0].select("span", attrs={"class": "unit-name  large"})[2].get_text()
    except IndexError:
        element = soup.find_all("div", attrs={"class": "pprice"})[0].find_all("span", attrs={"class":"unit-name red large"})[0].get_text()
    
    return element

In [92]:
volume = []
volume.append(get_volume(soup))
volume


[' / 0.75 L btl ']

In [87]:
soup.find_all("div", attrs={"class": "quantity-widget large has-0-flags"})[0]

IndexError: list index out of range

#### Biodinámico

In [97]:
link = "https://www.vinissimus.com/en/wine/la-montesa/"

request = requests.get(link)
soup = BeautifulSoup(request.content, 'html.parser')

In [98]:
info_dct = obtain_wine_info(soup)
new_df = pd.DataFrame.from_dict(info_dct)
new_df["link"] = link
new_df

,name,price,bottle,image,biodynamic,type,region,grapes,producer,alergens,alcohol_content,production,material,aging_period,casks_age,casks_wood,more_winemaking_info,vineyard_age,soil_type,climate,orientation,surface,wine_description,tasting,serving_storage_conditions,winemaking_info,vineyards,serve_temperature,food_pairing,customer_reviews,customer_reviews_number,last_score_parker,last_score_penin,last_score_year,link
0,La Montesa 2019,£16.00,/ 0.75 L btl,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,ORGANIC,Red wine,Rioja (Spain),100% Garnacha,Bodegas Palacios Remondo,Contains sulfites,14.5%,"650,000 - 700,000 bottles",Stainless steel,12 months,French and American oak,Bottled unfined and unfiltered,,Calcareous-clay / Pebbles,Mediterranean with Atlantic influence,Low yields,100.00 hectares,550.00 meters,With each sip Finca La Montesa shows that it i...,Its particularly transparent ruby red colour i...,We recommend serving the wine between 14 and 1...,A fresh and spicy character makes it the perfe...,"Grapes are harvested by hand, discarding exces...",Serve between 14ºC and 16ºC,Sausages and hamRicePoultry and white meats,4.3/5,292 reviews,93 PK,,2020,https://www.vinissimus.com/en/wine/la-montesa/


#### Párrafos

In [115]:
link = "https://www.vinissimus.com/en/wine/ultreia-saint-jacques/"

request = requests.get(link)
soup = BeautifulSoup(request.content, 'html.parser')

In [ ]:
wine_descr = []
tasting_descr = []
conditions_descr = []
winemake_descr = []
vineyards_descr = []
broad_text = [wine_descr, tasting_descr, conditions_descr, winemake_descr, vineyards_descr]

for descr in range(0, len(broad_text)):
    broad_text[descr].append(soup.find_all("div", attrs={"class": "sanitize-html"})[descr].get_text())

In [107]:
for i in soup.find_all("div", attrs={"class": "content"}):
    print(i.select("div", attrs={"class": "sanitize-html "}))

[<div class="product-details wine"><div class="product-image-wrapper has-flag"><div class="flags"><span class="flag horizontal">Recommended</span></div><div class="product-image desktop"><img alt="Ultreia Saint Jacques 2021" class="" height="500" src="https://cdn.vinissimus.com/img/unsafe/p500x/plain/local:///prfmtgrande/vi/ultsj20_anv800_1662703943.png" style="min-height: 5px; min-width: 5px;" width="170"/></div></div><div class="details"><div><div style="display: flex; justify-content: space-between; align-items: flex-start; width: 100%;"><div class="product-title"><h1>Ultreia Saint Jacques</h1><span class="heading fs-2em"> 2021</span></div><button aria-label="Add to my whishlist" class="likes styles_heartButton__aYIFq styles_hasLikes__TaEmI" title="Add to my whishlist"><i class="icon icon-heart-empty"></i><div class="styles_likes__Jvb7B">334</div></button></div><div class="slogan">The best introduction to Bierzo</div><div class="opinions-desktop" mincount="0"><div class="styles_opin

In [108]:
soup.find_all("h2", attrs={"class": "heading"})

[<h2 class="heading">Spec sheet</h2>,
 <h2 class="heading"><span>The wine</span><i class="icon icon-wine" style="font-size: 45px;"></i></h2>,
 <h2 class="heading"><span>What does this wine taste like?</span><i class="icon icon-taste"></i></h2>,
 <h2 class="heading"><span>Drinking and storing</span><i class="icon icon-service"></i></h2>,
 <h2 class="heading"><span>Food pairing</span><i class="icon icon-marriage"></i></h2>,
 <h2 class="heading"><span>Ratings and awards</span><i class="icon icon-score"></i></h2>,
 <h2 class="heading accordion">Wine critics reviews<small>Reviews by Jancis Robinson and Parker</small></h2>,
 <h2 class="heading"><span>Customer reviews</span><i class="icon icon-opinions" style="font-size: 40px;"></i></h2>,
 <h2 class="heading accordion"><span>Winemaking</span><i class="icon icon-elaboration"></i></h2>,
 <h2 class="heading accordion"><span>Vineyards</span><i class="icon icon-vineyard"></i></h2>]

In [126]:
dom = etree.HTML(str(soup))
print(dom.xpath('//*[@id="__next"]/main/div[5]/section/div[3]/div/table[2]/tbody/tr[1]').text)

AttributeError: 'list' object has no attribute 'text'

#### Notas de cata

In [91]:
link = "https://www.vinissimus.com/en/wine/aalto/"

request = requests.get(link)
soup = BeautifulSoup(request.content, 'html.parser')

In [20]:
for i in soup.find_all("table", attrs={"class":"styles_table__jWPKo info-grid taste"}):
    for j in i.find_all("td"):
        print(j.get_text())

Dark / Bright / Opaque
Fruit aromas / Liquorice / Vanilla / Coffee / Tobacco / Graphite
Intense / Fruit-forward / Creamy / Corpulent


In [92]:
soup.find_all("table", attrs={"class":"styles_table__jWPKo info-grid taste"})[0].find_all("td")

[<td>Dark / Bright / Opaque</td>,
 <td>Fruit aromas / Liquorice / Vanilla / Coffee / Tobacco / Graphite</td>,
 <td>Intense / Fruit-forward / Creamy / Corpulent</td>]

In [93]:
soup.find_all("table", attrs={"class":"styles_table__jWPKo info-grid taste"})[0]

<table class="styles_table__jWPKo info-grid taste"><tbody><tr><th class="auto-th"><div>View</div><i class="icon icon-eye" style="font-size: 24px;"></i></th><td>Dark / Bright / Opaque</td></tr><tr><th class="auto-th"><div>Bouquet</div><i class="icon icon-nose" style="font-size: 24px;"></i></th><td>Fruit aromas / Liquorice / Vanilla / Coffee / Tobacco / Graphite</td></tr><tr><th class="auto-th"><div>Mouth</div><i class="icon icon-mouth" style="font-size: 24px;"></i></th><td>Intense / Fruit-forward / Creamy / Corpulent</td></tr></tbody></table>

In [3]:
view = []
bouquet = []
mouth = []
tasting = [view, bouquet, mouth]

In [50]:
def get_table(lst, indexes):
    try:
        element = lst[indexes[0]]
    except IndexError:
        return ""
    else:
        if len(lst) == 1:
            print(indexes)
            print(f"indexes = {indexes[0]}")
            return lst[0].find_all("td")[indexes[0]].get_text()
        else:
            try:
                td_element = element.select("td")[indexes[1]]
            except IndexError:
                return ""
            else:
                return td_element.get_text()
            
# no funciona con tasting, sí funciona con tech info

In [32]:
wine_type = []
region = []
grapes = []
producer = []
alergens = []
alcohol = []
production = []
material = []
aging_period = []
casks_age = []
casks_wood = []
winemake = []
vineyard_age = []
soil_type = []
climate = []
orientation = []
surface = []

spec_sheet = [wine_type, region, grapes, producer, alergens, alcohol, production]
winemaking = [material, aging_period, casks_age, casks_wood, winemake]
vineyards = [vineyard_age, soil_type, climate, orientation, surface]
technical_info = [spec_sheet, winemaking, vineyards]

for info1 in range(0, len(technical_info)):
    for info2 in range(0, len(technical_info[info1])):
        technical_info[info1][info2].append(get_table(soup.find_all("table", attrs={"class": "styles_table__jWPKo info-grid technical-info"}), [info1, info2]))
technical_info

[[['Red wine'],
  ['Ribera del Duero (Spain)'],
  ['100% Tempranillo'],
  ['Aalto'],
  ['Contains sulfites'],
  ['15.0%'],
  ['250,000 - 300,000  bottles']],
 [['18 months'],
  ['New and semi-new'],
  ['French and American oak'],
  [''],
  ['']],
 [['Between 40 and 80 years'],
  ['Calcareous-clay / Sand / Stony'],
  ['Continental'],
  ['Very low yields'],
  ['']]]

In [57]:
view = []
bouquet = []
mouth = []
tasting = [view, bouquet, mouth]

for info in range(len(tasting)):
    tasting[info].append(soup.find_all("table", attrs={"class":"styles_table__jWPKo info-grid taste"})[0].find_all("td")[info].get_text())
tasting

[['Dark / Bright / Opaque'],
 ['Fruit aromas / Liquorice / Vanilla / Coffee / Tobacco / Graphite'],
 ['Intense / Fruit-forward / Creamy / Corpulent']]

In [87]:
def get_tasting(lst, index):
    try:
        element = lst[0]
    except IndexError:
        return ""
    else:
        return element.find_all("td")[index].get_text()

In [85]:
soup.find_all("table", attrs={"class":"styles_table__jWPKo info-grid taste"})[0]

IndexError: list index out of range

In [88]:
view = []
bouquet = []
mouth = []
tasting = [view, bouquet, mouth]

for i in range(len(tasting)):
    tasting[i].append(get_tasting(soup.find_all("table", attrs={"class":"styles_table__jWPKo info-grid taste"}), i))
tasting

[['Dark / Bright / Opaque'],
 ['Fruit aromas / Liquorice / Vanilla / Coffee / Tobacco / Graphite'],
 ['Intense / Fruit-forward / Creamy / Corpulent']]

In [ ]:
# el problema está en la función get_table en el momento en que mete el índice, pone 0 pero 1, 2 no ¿?¿¿?¿

In [41]:
tasting

[['Dark / Bright / Opaque'], [''], ['']]

In [19]:
len(soup.find_all("table", attrs={"class": "styles_table__jWPKo info-grid technical-info"}))

3

In [53]:
tasting

[['Dark / Bright / Opaque'], [''], ['']]

In [64]:
# Chat gpt (que no funciona si no está el elemento)

view = []
bouquet = []
mouth = []
tasting = [view, bouquet, mouth]

tables = soup.find_all("table", attrs={"class": "styles_table__jWPKo info-grid taste"})

for info in range(len(tasting)):
    rows = tables[0].find_all("tr")
    if len(rows) > info:
        data = rows[info].find_all("td")
        if len(data) > 0:
            tasting[info].append(data[0].get_text())
        else:
            tasting[info].append("")
    else:
        tasting[info].append("")


In [65]:
tasting

[['Intense'],
 ['Raspberry / Violets'],
 ['Red fruit / Mineral / Voluminous / Medium-bodied']]

In [89]:
# Function for obtaining all the info (with technical spec + exception handling + notas cata)

def obtain_wine_info(soup):
    
    # Define lists for each feature I want to obtain
    name = []
    price = []
    volume = []
    image = []
    agriculture = []
    
    wine_type = []
    region = []
    grapes = []
    producer = []
    alergens = []
    alcohol = []
    production = []
    material = []
    aging_period = []
    casks_age = []
    casks_wood = []
    winemake = []
    vineyard_age = []
    soil_type = []
    climate = []
    orientation = []
    surface = []

    spec_sheet = [wine_type, region, grapes, producer, alergens, alcohol, production]
    winemaking = [material, aging_period, casks_age, casks_wood, winemake]
    vineyards = [vineyard_age, soil_type, climate, orientation, surface]
    technical_info = [spec_sheet, winemaking, vineyards]
    
    wine_descr = []
    tasting_descr = []
    conditions_descr = []
    winemake_descr = []
    vineyards_descr = []
    broad_text = [wine_descr, tasting_descr, conditions_descr, winemake_descr, vineyards_descr]
    
    serve_temp = []
    food_pairing = []
    customer_reviews = []
    customer_reviews_number = []
    last_score_parker = []
    last_score_penin = []
    last_score_year = []
    
    view = []
    bouquet = []
    mouth = []
    tasting = [view, bouquet, mouth]
    
    # Assing elements to each list
    name.append(soup.find_all("div", attrs={"class": "product-title"})[0].get_text()) 
    price.append(get_price(soup.find_all("p", attrs={"class": "price uniq large"}), 0)) 
    volume.append(get_volume(soup))
    image.append(soup.find_all("div", attrs={"class": "product-image desktop"})[0].find("img").get("src")) 
    agriculture.append(get_element(soup.find_all("div", attrs={"class": "badge-with-label-text large"}), 0))
    
    for i in range(len(tasting)):
        tasting[i].append(get_tasting(soup.find_all("table", attrs={"class":"styles_table__jWPKo info-grid taste"}), i))
    
    for i in range(len(technical_info)):
        for j in range(len(technical_info[i])):
            technical_info[i][j].append(get_technical_info(soup.find_all("table", attrs={"class": "styles_table__jWPKo info-grid technical-info"}), i, j))
    
    for i in range(len(broad_text)):
        broad_text[i].append(get_element(soup.find_all("div", attrs={"class": "sanitize-html"}), i))

    element_list = soup.find_all("div", attrs={"style": "display: flex; justify-content: center; flex-direction: column;"})
    serve_temp.append(get_element(element_list, 0))
    
    element_list = soup.find_all("div", attrs={"style": "display: flex; flex-wrap: wrap; gap: 10px; line-height: 18px; font-size: 11px; text-transform: uppercase;"})
    food_pairing.append(get_element(element_list, 0))    
    
    customer_reviews.append(soup.find_all("div", attrs={"class": "styles_firstBlock__NtfrI"})[1].get_text())
    customer_reviews_number.append(soup.find_all("div", attrs={"class": "opinion-widget rating"})[0].find("span", attrs={"class":"styles_numOpinions__t_p9L"}).get_text())
    
    element_list = soup.find_all("table", attrs={"class": "styles_table__jWPKo info-grid score-awards"})
    last_score_parker.append(get_score(element_list, 0, "Parker"))
    last_score_penin.append(get_score(element_list, 0, "Peñín"))
    last_score_year.append(get_score_year(element_list, 0))

    # Create dictionary that stores all elements
    dct = {
        "name": name,
        "price": price,
        "bottle": volume,
        "type_wine": wine_type,
        "region": region,
        "grapes": grapes,
        "producer": producer,
        "wine_description": wine_descr,
        "tasting_descr": tasting_descr,
        "tasting_view": view,
        "tasting_bouquet": bouquet, 
        "tasting_mouth": mouth, 
        "serving_storage_conditions": conditions_descr,  
        "serve_temperature": serve_temp,
        "food_pairing": food_pairing,
        "alergens": alergens,
        "alcohol_content": alcohol,
        "production": production,
        "winemaking_info": winemake_descr,
        "vessel_material": material,
        "aging_period": aging_period,
        "casks_age": casks_age,
        "casks_wood": casks_wood,
        "more_winemaking_info": winemake,
        "vineyards": vineyards_descr,
        "type_agriculture": agriculture,
        "vineyard_age": vineyard_age,
        "soil_type": soil_type,
        "climate": climate,
        "orientation": orientation,
        "surface": surface,
        "customer_reviews": customer_reviews,
        "customer_reviews_number": customer_reviews_number,
        "last_score_parker": last_score_parker,
        "last_score_penin": last_score_penin,
        "last_score_year": last_score_year,
        "image": image
    }

    return dct

In [90]:
info_dct = obtain_wine_info(soup)
new_df = pd.DataFrame.from_dict(info_dct)
new_df["url"] = link
new_df

,name,price,bottle,type_wine,region,grapes,producer,wine_description,tasting_descr,tasting_view,tasting_bouquet,tasting_mouth,serving_storage_conditions,serve_temperature,food_pairing,alergens,alcohol_content,production,winemaking_info,vessel_material,aging_period,casks_age,casks_wood,more_winemaking_info,vineyards,type_agriculture,vineyard_age,soil_type,climate,orientation,surface,customer_reviews,customer_reviews_number,last_score_parker,last_score_penin,last_score_year,image,url
0,Aalto 2020 (Magnum),£87.35,/ 1.5 L btl,Red wine,Ribera del Duero (Spain),100% Tempranillo,Aalto,Aalto is one of the Ribera del Duero’s great b...,Aalto is the emblematic name of a winery which...,Dark / Bright / Opaque,Fruit aromas / Liquorice / Vanilla / Coffee / ...,Intense / Fruit-forward / Creamy / Corpulent,,Serve between 16ºC and 18ºC,Stews and casserolesSausages and hamRed meat,Contains sulfites,15.0%,"250,000 - 300,000 bottles",,18 months,New and semi-new,French and American oak,,,,PARKER,Between 40 and 80 years,Calcareous-clay / Sand / Stony,Continental,Very low yields,,4.8/5,116 reviews,94 PK,,2020,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/aalto/


#### Más errores

In [78]:
link = "https://www.vinissimus.com/en/wine/anima-negra-an-2/"

request = requests.get(link)
soup = BeautifulSoup(request.content, 'html.parser')

In [79]:
info_dct = obtain_wine_info(soup)
new_df = pd.DataFrame.from_dict(info_dct)
new_df["url"] = link
new_df

IndexError: list index out of range

In [81]:
# Define lists for each feature I want to obtain
name = []
price = []
volume = []
image = []
agriculture = []

wine_type = []
region = []
grapes = []
producer = []
alergens = []
alcohol = []
production = []
material = []
aging_period = []
casks_age = []
casks_wood = []
winemake = []
vineyard_age = []
soil_type = []
climate = []
orientation = []
surface = []

spec_sheet = [wine_type, region, grapes, producer, alergens, alcohol, production]
winemaking = [material, aging_period, casks_age, casks_wood, winemake]
vineyards = [vineyard_age, soil_type, climate, orientation, surface]
technical_info = [spec_sheet, winemaking, vineyards]

wine_descr = []
tasting_descr = []
conditions_descr = []
winemake_descr = []
vineyards_descr = []
broad_text = [wine_descr, tasting_descr, conditions_descr, winemake_descr, vineyards_descr]

serve_temp = []
food_pairing = []
customer_reviews = []
customer_reviews_number = []
last_score_parker = []
last_score_penin = []
last_score_year = []

view = []
bouquet = []
mouth = []
tasting = [view, bouquet, mouth]

# Assing elements to each list
name.append(soup.find_all("div", attrs={"class": "product-title"})[0].get_text()) 
price.append(get_price(soup.find_all("p", attrs={"class": "price uniq large"}), 0)) 
volume.append(get_volume(soup))
image.append(soup.find_all("div", attrs={"class": "product-image desktop"})[0].find("img").get("src")) 
agriculture.append(get_element(soup.find_all("div", attrs={"class": "badge-with-label-text large"}), 0))

for info1 in range(len(technical_info)):
    for info2 in range(len(technical_info[info1])):
        technical_info[info1][info2].append(get_technical_info(soup.find_all("table", attrs={"class": "styles_table__jWPKo info-grid technical-info"}), info1, info2))

for descr in range(len(broad_text)):
    broad_text[descr].append(get_element(soup.find_all("div", attrs={"class": "sanitize-html"}), descr))

element_list = soup.find_all("div", attrs={"style": "display: flex; justify-content: center; flex-direction: column;"})
serve_temp.append(get_element(element_list, 0))

element_list = soup.find_all("div", attrs={"style": "display: flex; flex-wrap: wrap; gap: 10px; line-height: 18px; font-size: 11px; text-transform: uppercase;"})
food_pairing.append(get_element(element_list, 0))    

customer_reviews.append(soup.find_all("div", attrs={"class": "styles_firstBlock__NtfrI"})[1].get_text())
customer_reviews_number.append(soup.find_all("div", attrs={"class": "opinion-widget rating"})[0].find("span", attrs={"class":"styles_numOpinions__t_p9L"}).get_text())

element_list = soup.find_all("table", attrs={"class": "styles_table__jWPKo info-grid score-awards"})
last_score_parker.append(get_score(element_list, 0, "Parker"))
last_score_penin.append(get_score(element_list, 0, "Peñín"))
last_score_year.append(get_score_year(element_list, 0))

tables = soup.find_all("table", attrs={"class": "styles_table__jWPKo info-grid taste"})

for info in range(len(tasting)):
    rows = tables[0].find_all("tr")
    if len(rows) > info:
        data = rows[info].find_all("td")
        if len(data) > 0:
            tasting[info].append(data[0].get_text())
        else:
            tasting[info].append("")
    else:
        tasting[info].append("")

# Create dictionary that stores all elements
dct = {
    "name": name,
    "price": price,
    "bottle": volume,
    "type_wine": wine_type,
    "region": region,
    "grapes": grapes,
    "producer": producer,
    "wine_description": wine_descr,
    "tasting_descr": tasting_descr,
    "tasting_view": view,
    "tasting_bouquet": bouquet, 
    "tasting_mouth": mouth, 
    "serving_storage_conditions": conditions_descr,  
    "serve_temperature": serve_temp,
    "food_pairing": food_pairing,
    "alergens": alergens,
    "alcohol_content": alcohol,
    "production": production,
    "winemaking_info": winemake_descr,
    "vessel_material": material,
    "aging_period": aging_period,
    "casks_age": casks_age,
    "casks_wood": casks_wood,
    "more_winemaking_info": winemake,
    "vineyards": vineyards_descr,
    "type_agriculture": agriculture,
    "vineyard_age": vineyard_age,
    "soil_type": soil_type,
    "climate": climate,
    "orientation": orientation,
    "surface": surface,
    "customer_reviews": customer_reviews,
    "customer_reviews_number": customer_reviews_number,
    "last_score_parker": last_score_parker,
    "last_score_penin": last_score_penin,
    "last_score_year": last_score_year,
    "image": image
}

IndexError: list index out of range

#### Tasting percentages

In [2]:
link = "https://www.vinissimus.com/en/wine/blanc-pescador/"

request = requests.get(link)
soup = BeautifulSoup(request.content, 'html.parser')

In [13]:
for i in soup.find_all("div", attrs={"class":"styles_bar__Y54KE"}):
    print(i.find_all("span")[0])

<span class="styles_progress__ob9IL" style="--percent: 75%;"></span>
<span class="styles_progress__ob9IL" style="--percent: 70%;"></span>
<span class="styles_progress__ob9IL" style="--percent: 20%;"></span>
<span class="styles_progress__ob9IL" style="--percent: 70%;"></span>


In [3]:
def get_score_year(lst, index):
    try:
        lst[index]
    except IndexError:
        return ""
    else:
        return lst[index].find("th").get_text()    

In [4]:
light_bold = []
smooth_tannic = []
dry_sweet = []
soft_acidic = []
taste_percentages = [light_bold, smooth_tannic, dry_sweet, soft_acidic]

In [15]:
def get_percentages(lst, index):
    try: 
        element = lst[index]
    except IndexError:
        return ""
    else:
        return element.find_all("span")[0]

In [49]:
for i in range(len(taste_percentages)):
    element = str(get_percentages(soup.find_all("div", attrs={"class":"styles_bar__Y54KE"}), i))
    element = int(element[55:-11])
    taste_percentages[i].append(element)

In [50]:
taste_percentages

[['60', 60, 60], ['70', 70, 70], ['15', 15, 15], ['70', 70, 70]]

In [32]:
len('%;"></span>')

11

In [34]:
text = '<span class="styles_progress__ob9IL" style="--percent: 60%;"></span>'
text = text[55:-11]
text

'60'

In [6]:
def obtain_wine_info(soup):
    
    # Define lists for each feature I want to obtain
    name = []
    price = []
    volume = []
    image = []
    agriculture = []
    
    wine_type = []
    region = []
    grapes = []
    producer = []
    alergens = []
    alcohol = []
    production = []
    material = []
    aging_period = []
    casks_age = []
    casks_wood = []
    winemake = []
    vineyard_age = []
    soil_type = []
    climate = []
    orientation = []
    surface = []

    spec_sheet = [wine_type, region, grapes, producer, alergens, alcohol, production]
    winemaking = [material, aging_period, casks_age, casks_wood, winemake]
    vineyards = [vineyard_age, soil_type, climate, orientation, surface]
    technical_info = [spec_sheet, winemaking, vineyards]
    
    wine_descr = []
    tasting_descr = []
    conditions_descr = []
    winemake_descr = []
    vineyards_descr = []
    broad_text = [wine_descr, tasting_descr, conditions_descr, winemake_descr, vineyards_descr]
    
    serve_temp = []
    food_pairing = []
    customer_reviews = []
    customer_reviews_number = []
    last_score_parker = []
    last_score_penin = []
    last_score_year = []
    
    view = []
    bouquet = []
    mouth = []
    tasting = [view, bouquet, mouth]
    
    light_bold = []
    smooth_tannic = []
    dry_sweet = []
    soft_acidic = []
    taste_percentages = [light_bold, smooth_tannic, dry_sweet, soft_acidic]
    
    # Assing elements to each list
    name.append(soup.find_all("div", attrs={"class": "product-title"})[0].get_text()) 
    price.append(get_price(soup.find_all("p", attrs={"class": "price uniq large"}), 0)) 
    volume.append(get_volume(soup))
    image.append(soup.find_all("div", attrs={"class": "product-image desktop"})[0].find("img").get("src")) 
    agriculture.append(get_element(soup.find_all("div", attrs={"class": "badge-with-label-text large"}), 0))
    
    for i in range(len(tasting)):
        tasting[i].append(get_tasting(soup.find_all("table", attrs={"class":"styles_table__jWPKo info-grid taste"}), i))
        
    for i in range(len(taste_percentages)):
        element = str(get_percentages(soup.find_all("div", attrs={"class":"styles_bar__Y54KE"}), i))
        if element == "":
            taste_percentages[i].append(element)
        else:
            element = int(element[55:-11])
            taste_percentages[i].append(element)
    
    for i in range(len(technical_info)):
        for j in range(len(technical_info[i])):
            technical_info[i][j].append(get_technical_info(soup.find_all("table", attrs={"class": "styles_table__jWPKo info-grid technical-info"}), i, j))
    
    for i in range(len(broad_text)):
        broad_text[i].append(get_element(soup.find_all("div", attrs={"class": "sanitize-html"}), i))

    element_list = soup.find_all("div", attrs={"style": "display: flex; justify-content: center; flex-direction: column;"})
    serve_temp.append(get_element(element_list, 0))
    
    element_list = soup.find_all("div", attrs={"style": "display: flex; flex-wrap: wrap; gap: 10px; line-height: 18px; font-size: 11px; text-transform: uppercase;"})
    food_pairing.append(get_element(element_list, 0))    
    
    customer_reviews.append(soup.find_all("div", attrs={"class": "styles_firstBlock__NtfrI"})[1].get_text())
    customer_reviews_number.append(soup.find_all("div", attrs={"class": "opinion-widget rating"})[0].find("span", attrs={"class":"styles_numOpinions__t_p9L"}).get_text())
    
    element_list = soup.find_all("table", attrs={"class": "styles_table__jWPKo info-grid score-awards"})
    last_score_parker.append(get_score(element_list, 0, "Parker"))
    last_score_penin.append(get_score(element_list, 0, "Peñín"))
    last_score_year.append(get_score_year(element_list, 0))

    # Create dictionary that stores all elements
    dct = {
        "name": name,
        "price": price,
        "bottle": volume,
        "type_wine": wine_type,
        "region": region,
        "grapes": grapes,
        "producer": producer,
        "wine_description": wine_descr,
        "tasting_descr": tasting_descr,
        "tasting_view": view,
        "tasting_bouquet": bouquet, 
        "tasting_mouth": mouth, 
        "light_bold_perc": light_bold, 
        "smooth_tannic_perc": smooth_tannic,
        "dry_sweet_perc": dry_sweet,
        "soft_acidic_perc": soft_acidic,
        "serving_storage_conditions": conditions_descr,  
        "serve_temperature": serve_temp,
        "food_pairing": food_pairing,
        "alergens": alergens,
        "alcohol_content": alcohol,
        "production": production,
        "winemaking_info": winemake_descr,
        "vessel_material": material,
        "aging_period": aging_period,
        "casks_age": casks_age,
        "casks_wood": casks_wood,
        "more_winemaking_info": winemake,
        "vineyards": vineyards_descr,
        "type_agriculture": agriculture,
        "vineyard_age": vineyard_age,
        "soil_type": soil_type,
        "climate": climate,
        "orientation": orientation,
        "surface": surface,
        "customer_reviews": customer_reviews,
        "customer_reviews_number": customer_reviews_number,
        "last_score_parker": last_score_parker,
        "last_score_penin": last_score_penin,
        "last_score_year": last_score_year,
        "image": image
    }

    return dct

In [7]:
# Functions for exception handling when there is no element to retrieve

def get_element(lst, index):
    try:
        lst[index]
    except IndexError:
        return ""
    else:
        return lst[index].get_text()
    
def get_price(lst, index):
    try:
        element = lst[index].get_text()
    except IndexError:
        element = soup.find_all("div", attrs={"class": "pprice"})[0].find("p").get_text()
    
    return element

def get_volume(soup):
    try:
        element = soup.find_all("div", attrs={"class": "quantity-widget large has-0-flags"})[0].select("span", attrs={"class": "unit-name  large"})[2].get_text()
    except IndexError:
        element = soup.find_all("div", attrs={"class": "pprice"})[0].find_all("span", attrs={"class":"unit-name red large"})[0].get_text()
    
    return element

def get_tasting(lst, index):
    try:
        element = lst[0]
    except IndexError:
        return ""
    else:
        return element.find_all("td")[index].get_text()
    
def get_percentages(lst, index):
    try: 
        element = lst[index]
    except IndexError:
        return ""
    else:
        return element.find_all("span")[0]

def get_technical_info(lst, index1, index2):
    try:
        element = lst[index1]
    except IndexError:
        return ""
    else:
        try:
            td_element = element.select("td")[index2]
        except IndexError:
            return ""
        else:
            return td_element.get_text()
    
def get_score(lst, index, score_name):
    try:
        element = lst[index]
    except IndexError:
        return ""
    else:
        try: 
            td_element = element.find("td", attrs={"title": score_name})
        except AttributeError:
            return ""
        else:
            if td_element is not None:
                return td_element.get_text()
            else:
                return ""

def get_score_year(lst, index):
    try:
        lst[index]
    except IndexError:
        return ""
    else:
        return lst[index].find("th").get_text()  

In [8]:
info_dct = obtain_wine_info(soup)
new_df = pd.DataFrame.from_dict(info_dct)
new_df["url"] = link
new_df

,name,price,bottle,type_wine,region,grapes,producer,wine_description,tasting_descr,tasting_view,tasting_bouquet,tasting_mouth,light_bold_perc,smooth_tannic_perc,dry_sweet_perc,soft_acidic_perc,serving_storage_conditions,serve_temperature,food_pairing,alergens,alcohol_content,production,winemaking_info,vessel_material,aging_period,casks_age,casks_wood,more_winemaking_info,vineyards,type_agriculture,vineyard_age,soil_type,climate,orientation,surface,customer_reviews,customer_reviews_number,last_score_parker,last_score_penin,last_score_year,image,url
0,Blanc Pescador,£7.80,/ 0.75 L btl,Frizzante white wine,Wines without GI (Spain),61% Macabeo6% Parellada33% Xarel·lo,Perelada,Blanc Pescador and Cresta Rosa have been bench...,"A pale colour, with light yellow-green hues an...",Straw yellow / Pale / Greenish hues / Bright,Aromatic / Fruit aromas / Higly aromatic,Light / Pleasant / Sparkling,,,,,The prestigious and successful Perelada winery...,,Snacks and appetizersStarters and suppersSalad...,Contains sulfites,11.5%,,,,,,,,,,,,,,,4.4/5,45 reviews,,,,https://cdn.vinissimus.com/img/unsafe/p500x/pl...,https://www.vinissimus.com/en/wine/blanc-pesca...


#### NaNs in bottle column

In [5]:
bottle_nan = pd.read_csv("bottle_nan.csv")

In [6]:
bottle_nan_links = list(bottle_nan["url"])
bottle_nan_links

['https://www.vinissimus.com/en/wine/aalto/',
 'https://www.vinissimus.com/en/wine/anima-negra-an-2/',
 'https://www.vinissimus.com/en/wine/vina-tondonia-blanco-reserva/',
 'https://www.vinissimus.com/en/wine/karman-tinto/',
 'https://www.vinissimus.com/en/wine/salinas-mo-monastrell/',
 'https://www.vinissimus.com/en/wine/vina-tondonia-gran-reserva-historico/',
 'https://www.vinissimus.com/en/wine/blanco-nieva-sauvignon/',
 'https://www.vinissimus.com/en/wine/tardencuba-autor/',
 'https://www.vinissimus.com/en/wine/vella-lola-rosat/',
 'https://www.vinissimus.com/en/wine/malpendre/',
 'https://www.vinissimus.com/en/cava/stars-brut-reserva/',
 'https://www.vinissimus.com/en/cava/argila-rose-reserva-bn/',
 'https://www.vinissimus.com/en/wine/torre-del-veguer-jeronimus/',
 'https://www.vinissimus.com/en/cava/parxet-brut-reserva/',
 'https://www.vinissimus.com/en/wine/l-enclos-de-peralba-tres-feixes/',
 'https://www.vinissimus.com/en/cava/mont-marcal-reserva-brut/',
 'https://www.vinissimu

In [13]:
link = bottle_nan_links[2]

request = requests.get(link)
soup = BeautifulSoup(request.content, 'html.parser')

In [22]:
soup.find_all("div", attrs={"class":"quantity-widget large has-0-flags"})[0].select("span", attrs={"class":"unit-name  large"})[1].get_text()

'£'

In [18]:
bottle = pd.DataFrame()
for i in soup.find_all("div", attrs={"class":"quantity-widget large has-0-flags"})[0].select("span", attrs={"class":"unit-name  large"}):
    element = i.get_text()
    if "btl" in element:
        dict = {"url": link, "bottle": [element]}
        new_df = pd.DataFrame.from_dict(dict)
        bottle = pd.concat([bottle, new_df])    
        
bottle   

,url,bottle
0,https://www.vinissimus.com/en/wine/vina-tondon...,/ 0.75 L btl


In [21]:
# bottle = pd.DataFrame()

# while True:
#     try:
#         for link in bottle_nan_links:
#             request = requests.get(link)
#             soup = BeautifulSoup(request.content, 'html.parser')
#             for i in soup.find_all("div", attrs={"class": "quantity-widget large has-0-flags"})[0].select("span", attrs={"class": "unit-name  large"}):
#                 element = i.get_text()
#                 print(element)
#                 if "btl" in element:
#                     dict = {"url": link, "bottle": [element]}
#                     new_df = pd.DataFrame.from_dict(dict)
#                     bottle = pd.concat([bottle, new_df])
                    
#             wait_time = np.random.randint(3, 7)
#             sleep(wait_time)
#     except Exception as e:
#         continue
#     break
    
    
# bottle.to_csv("bottle0907.csv", index=False)

£
 / 0.75 L btl 

Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
i
£
 / 0.75 L btl 
1 bottle

£
2 bottles

£
3 bottles

£
-+

£
Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
i
£
 / 0.75 L btl 
1 bottle

£
2 bottles

£
3 bottles

£
-+

£
Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
i
£
 / 0.75 L btl 
1 bottle

£
2 bottles

£
3 bottles

£
4 bottles

£
Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
£
 

i
£
 / 0.75 L btl 
1 bottle

£
2 bottles

£
3 bottles

£
-+

£
Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
i
£
 / 0.75 L btl 
1 bottle

£
2 bottles

£
3 bottles

£
4 bottles

£
Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
i
£
 / 0.75 L btl 
1 bottle

£
2 bottles

£
3 bottles

£
-+

£
Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
i
£
 / 0.75 L btl 
1 bottle

£
2 bottles

£
3 bottles

£
-+

£
Insurance included & returns guarantee
£
 / 0.75 L btl

£
 / 0.75 L btl 

Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
i
£
 / 0.75 L btl 
1 bottle

£
2 bottles

£
3 bottles

£
-+

£
Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
i
£
 / 0.75 L btl 
1 bottle

£
2 bottles

£
3 bottles

£
-+

£
Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
i
£
 / 0.75 L btl 
1 bottle

£
2 bottles

£
3 bottles

£
4 bottles

£
Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
£
 

£
 / 0.75 L btl 

Insurance included & returns guarantee
i
£
 / 0.75 L btl 
1 bottle

£
2 bottles

£
3 bottles

£
-+

£
Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
i
£
 / 0.75 L btl 
1 bottle

£
2 bottles

£
3 bottles

£
-+

£
Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
i
£
 / 0.75 L btl 
1 bottle

£
2 bottles

£
3 bottles

£
4 bottles

£
Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
i
£
 / 0.75 L btl 
1 bottle

£
2 bottles

£
3 bottles

£
-+


£
 / 0.75 L btl 

Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
i
£
 / 0.75 L btl 
1 bottle

£
2 bottles

£
3 bottles

£
-+

£
Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & retu

...

£
 / 0.75 L btl 

Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
i
£
 / 0.75 L btl 
1 bottle

£
2 bottles

£
3 bottles

£
-+

£
Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
i
£
 / 0.75 L btl 
1 bottle

£
2 bottles

£
3 bottles

£
-+

£
Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
i
£
 / 0.75 L btl 
1 bottle

£
2 bottles

£
3 bottles

£
4 bottles

£
Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
£
 

£
 / 0.75 L btl 

Insurance included & returns guarantee
i
£
 / 0.75 L btl 
1 bottle

£
2 bottles

£
3 bottles

£
-+

£
Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
i
£
 / 0.75 L btl 
1 bottle

£
2 bottles

£
3 bottles

£
4 bottles

£
Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
i
£
 / 0.75 L btl 
1 bottle

£
2 bottles

£
3 bottles

£
-+

£
Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
i
£
 / 0.75 L btl 
1 bottle

£
2 bottles

£
3 bottles

£
-+


£
 / 0.75 L btl 

Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
i
£
 / 0.75 L btl 
1 bottle

£
2 bottles

£
3 bottles

£
4 bottles

£
Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
i
£
 / 0.75 L btl 
1 bottle

£
2 bottles

£
3 bottles

£
-+

£
Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
i
£
 / 0.75 L btl 
1 bottle

£
2 bottles

£
3 bottles

£
-+

£
Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
i
£

£
 / 0.75 L btl 

Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
i
£
 / 0.75 L btl 
1 bottle

£
2 bottles

£
3 bottles

£
-+

£
Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
i
£
 / 0.75 L btl 
1 bottle

£
2 bottles

£
3 bottles

£
-+

£
Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
i
£
 / 0.75 L btl 
1 bottle

£
2 bottles

£
3 bottles

£
4 bottles

£
Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
£
 / 0.75 L btl 

Insurance included & returns guarantee
£
 

KeyboardInterrupt: 

In [26]:
bottle = pd.DataFrame()
wrong_links = []

for link in bottle_nan_links:
    try:
        request = requests.get(link)
        soup = BeautifulSoup(request.content, 'html.parser')
        wait_time = np.random.randint(3, 7)
        sleep(wait_time)
        for i in soup.find_all("div", attrs={"class": "quantity-widget large has-0-flags"})[0].select("span", attrs={"class": "unit-name  large"}):
            element = i.get_text()
            if "btl" in element:
                dict = {"url": link, "bottle": [element]}
                new_df = pd.DataFrame.from_dict(dict)
                bottle = pd.concat([bottle, new_df])
    except Exception as e:
        wrong_links.append(link)
        print(link)
        print(e)  
        
          
bottle.to_csv("bottle1107.csv", index=False)

https://www.vinissimus.com/en/cava/parxet-brut-reserva/
list index out of range
https://www.vinissimus.com/en/cava/mont-marcal-reserva-brut/
list index out of range
https://www.vinissimus.com/en/wine/congo/
HTTPSConnectionPool(host='www.vinissimus.com', port=443): Max retries exceeded with url: /en/wine/congo/ (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000002CFDB35DE50>: Failed to establish a new connection: [WinError 10060] Se produjo un error durante el intento de conexión ya que la parte conectada no respondió adecuadamente tras un periodo de tiempo, o bien se produjo un error en la conexión establecida ya que el host conectado no ha podido responder'))
https://www.vinissimus.com/en/cava/guspira-estones-de-mishima/
list index out of range
https://www.vinissimus.com/en/wine/michelini-i-mufatto-el-rapolao/
HTTPSConnectionPool(host='www.vinissimus.com', port=443): Max retries exceeded with url: /en/wine/michelini-i-mufatto-el-rapolao/ (Caused by NewC

https://www.vinissimus.com/en/wine/abadia-de-cova-mencia-barrica/
HTTPSConnectionPool(host='www.vinissimus.com', port=443): Max retries exceeded with url: /en/wine/abadia-de-cova-mencia-barrica/ (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000002CFD9C77040>: Failed to establish a new connection: [WinError 10060] Se produjo un error durante el intento de conexión ya que la parte conectada no respondió adecuadamente tras un periodo de tiempo, o bien se produjo un error en la conexión establecida ya que el host conectado no ha podido responder'))
https://www.vinissimus.com/en/wine/a-mi-manera/
HTTPSConnectionPool(host='www.vinissimus.com', port=443): Max retries exceeded with url: /en/wine/a-mi-manera/ (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000002CFDB5EA040>: Failed to establish a new connection: [WinError 10060] Se produjo un error durante el intento de conexión ya que la parte conectada no respondió adecuadamente 

https://www.vinissimus.com/en/wine/mas-bertran-ancestral/
('Connection aborted.', TimeoutError(10060, 'Se produjo un error durante el intento de conexión ya que la parte conectada no respondió adecuadamente tras un periodo de tiempo, o bien se produjo un error en la conexión establecida ya que el host conectado no ha podido responder', None, 10060, None))
https://www.vinissimus.com/en/wine/clamor-xarel-lo/
HTTPSConnectionPool(host='www.vinissimus.com', port=443): Max retries exceeded with url: /en/wine/clamor-xarel-lo/ (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000002CFDB610CD0>: Failed to establish a new connection: [WinError 10060] Se produjo un error durante el intento de conexión ya que la parte conectada no respondió adecuadamente tras un periodo de tiempo, o bien se produjo un error en la conexión establecida ya que el host conectado no ha podido responder'))
https://www.vinissimus.com/en/wine/valpincia-reserva/
HTTPSConnectionPool(host='www.vi

https://www.vinissimus.com/en/wine/gran-feudo-tempranillo-rioja/
HTTPSConnectionPool(host='www.vinissimus.com', port=443): Max retries exceeded with url: /en/wine/gran-feudo-tempranillo-rioja/ (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000002CFD9598490>: Failed to establish a new connection: [WinError 10060] Se produjo un error durante el intento de conexión ya que la parte conectada no respondió adecuadamente tras un periodo de tiempo, o bien se produjo un error en la conexión establecida ya que el host conectado no ha podido responder'))
https://www.vinissimus.com/en/wine/el-quintanal/
HTTPSConnectionPool(host='www.vinissimus.com', port=443): Max retries exceeded with url: /en/wine/el-quintanal/ (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000002CFD94DA580>: Failed to establish a new connection: [WinError 10060] Se produjo un error durante el intento de conexión ya que la parte conectada no respondió adecuadamente 

In [27]:
len(wrong_links)

56

In [28]:
df = pd.DataFrame(wrong_links)
df.to_csv("bottle_wrong_links1107.csv", index=False)

### Rioja alavesa

In [3]:
link = "https://riojawine.com/bodegas-rioja/"

request = requests.get(link)
soup = BeautifulSoup(request.content, 'html.parser')

In [4]:
soup.find_all("span", attrs={"class":"__name"})

[<span class="__name">Grandes Vinos F. Olimpia</span>,
 <span class="__name">VIÑEDOS RUIZ JIMENEZ</span>,
 <span class="__name">Bodegas Abeica</span>,
 <span class="__name">BODEGAS ONTAÑÓN</span>,
 <span class="__name">Bodega Señorío de las Viñas</span>,
 <span class="__name">Bodegas y Viñedos Casado Morales</span>,
 <span class="__name">CUNA DE REYES</span>,
 <span class="__name">Arizcuren</span>,
 <span class="__name">Alegre Valgañón</span>,
 <span class="__name">BODEGAS VALENCISO</span>,
 <span class="__name">Bodegas Marqués del Puerto</span>,
 <span class="__name">BODEGAS Y VIÑEDOS ORTEGA EZQUERRO</span>,
 <span class="__name">Bodegas Afersa</span>,
 <span class="__name">BODEGAS PACO GARCIA </span>,
 <span class="__name">BODEGAS VALLOBERA</span>,
 <span class="__name">Bodegas Baigorri</span>,
 <span class="__name">BODEGA SEÑORIO DE LA ESTRELLA</span>,
 <span class="__name">Bodegas García de Olano</span>,
 <span class="__name">BODEGAS RUIZ DE VIÑASPRE</span>,
 <span class="__name">O

In [10]:
bottle_nan_links2 = ['https://www.vinissimus.com/en/wine/aalto/',
 'https://www.vinissimus.com/en/wine/anima-negra-an-2/',
 'https://www.vinissimus.com/en/wine/vina-tondonia-blanco-reserva/']